In [2]:
import requests
import pandas as pd
from pathlib import Path
import json


# Set up output directory
data_dir = Path("../data")
data_dir.mkdir(parents=True, exist_ok=True)

# Example location (e.g., Nashville, TN)
latitude = 36.1627
longitude = -86.7816

# NASA POWER API base URL for climatology (daily averages)
url = (
    f"https://power.larc.nasa.gov/api/temporal/daily/point"
    f"?parameters=T2M_MAX,T2M_MIN,PRECTOTCORR"
    f"&community=AG"
    f"&longitude={longitude}&latitude={latitude}"
    f"&start=20220101&end=20231231"
    f"&format=JSON"
)

response = requests.get(url)
data = response.json()

# Preview the keys
list(data.keys())


['type', 'geometry', 'properties', 'header', 'messages', 'parameters', 'times']

In [3]:
import json

# Save the API response to a JSON file
with open(data_dir / "nasa_power_response.json", "w") as f:
    json.dump(data, f, indent=2)

In [4]:
#data['parameters'].keys()

data['parameters']['PRECTOTCORR']



{'units': 'mm/day', 'longname': 'Precipitation Corrected'}

In [5]:
# Extract the parameter data
params = data['properties']['parameter']

# Get all dates (keys) from one of the parameters
dates = list(params['T2M_MAX'].keys())

# Build a DataFrame
df = pd.DataFrame({
    'date': pd.to_datetime(dates, format='%Y%m%d'),
    'T2M_MAX': [params['T2M_MAX'][d] for d in dates],
    'T2M_MIN': [params['T2M_MIN'][d] for d in dates],
    'PRECTOTCORR': [params['PRECTOTCORR'][d] for d in dates],
})

# Set the date as the index
df.set_index('date', inplace=True)

# Preview the DataFrame
df.head()

,T2M_MAX,T2M_MIN,PRECTOTCORR
date,,,
2022-01-01,23.35,12.98,33.73
2022-01-02,11.74,-0.71,9.67
2022-01-03,1.98,-5.26,0.82
2022-01-04,5.66,-6.86,0.54
2022-01-05,10.60,-1.61,0.93


In [6]:
# Add year and day_of_year columns
df['year'] = df.index.year
df['day_of_year'] = df.index.dayofyear

# Preview the DataFrame
df.head()

,T2M_MAX,T2M_MIN,PRECTOTCORR,year,day_of_year
date,,,,,
2022-01-01,23.35,12.98,33.73,2022,1
2022-01-02,11.74,-0.71,9.67,2022,2
2022-01-03,1.98,-5.26,0.82,2022,3
2022-01-04,5.66,-6.86,0.54,2022,4
2022-01-05,10.60,-1.61,0.93,2022,5


In [8]:
import plotly.express as px

# Group by day_of_year and calculate the average T2M_MAX
avg_t2m_max = df.groupby('day_of_year')['T2M_MIN'].mean().reset_index()

# Create interactive plot
fig = px.line(
    avg_t2m_max,
    x='day_of_year',
    y='T2M_MIN',
    title='Average T2M_MAX by Day of Year',
    labels={'day_of_year': 'Day of Year', 'T2M_MAX': 'Average T2M_MAX (°C)'}
)
fig.show()